In [6]:
!nvidia-smi

Mon Jan 12 13:28:13 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Задание 1


    Реализуйте программу на CUDA для поэлементной обработки массива (например,
    умножение каждого элемента на число).

    Реализуйте две версии программы:
      1. с использованием только глобальной памяти;
      2. с использованием разделяемой памяти.
      Сравните время выполнения обеих реализаций для массива размером 1 000 000
      элементов.

In [11]:
%%writefile task1.cu

#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

// -------------------------------
// CUDA kernel: версия с глобальной памятью
// -------------------------------
__global__ void multiplyGlobal(int* d_arr, int factor, int n) {

    // Вычисляем глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверяем, чтобы не выйти за границы массива
    if (idx < n) {
        // Каждый поток умножает свой элемент массива
        d_arr[idx] *= factor;
    }
}

// -------------------------------
// CUDA kernel: версия с разделяемой памятью
// -------------------------------
__global__ void multiplyShared(int* d_arr, int factor, int n) {

    // Объявляем разделяемую память для блока
    __shared__ int sharedArr[256];

    // Глобальный индекс потока
    int globalIdx = blockIdx.x * blockDim.x + threadIdx.x;

    // Локальный индекс внутри блока
    int localIdx = threadIdx.x;

    // Загружаем данные из глобальной памяти в shared memory
    if (globalIdx < n) {
        sharedArr[localIdx] = d_arr[globalIdx];
    }

    // Синхронизация всех потоков блока
    __syncthreads();

    // Выполняем вычисления в shared memory
    if (globalIdx < n) {
        sharedArr[localIdx] *= factor;
    }

    // Снова синхронизация
    __syncthreads();

    // Записываем результат обратно в глобальную память
    if (globalIdx < n) {
        d_arr[globalIdx] = sharedArr[localIdx];
    }
}

int main() {

    // Размер массива
    const int N = 1'000'000;

    // Размер массива в байтах
    size_t size = N * sizeof(int);

    // Множитель
    int factor = 2;

    // Выделяем память на хосте
    int* h_arr = new int[N];

    // Инициализируем массив
    for (int i = 0; i < N; i++) {
        h_arr[i] = i;
    }

    // Выделяем память на GPU
    int* d_arr;
    cudaMalloc(&d_arr, size);

    // Копируем данные с CPU на GPU
    cudaMemcpy(d_arr, h_arr, size, cudaMemcpyHostToDevice);

    // Параметры запуска ядра
    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    // -------------------------------
    // Запуск версии с глобальной памятью
    // -------------------------------
    auto startGlobal = high_resolution_clock::now();

    multiplyGlobal<<<blocksPerGrid, threadsPerBlock>>>(d_arr, factor, N);

    // Ожидаем завершения ядра
    cudaDeviceSynchronize();

    auto endGlobal = high_resolution_clock::now();

    double timeGlobal =
        duration<double, milli>(endGlobal - startGlobal).count();

    // -------------------------------
    // Подготовка данных для shared версии
    // -------------------------------
    cudaMemcpy(d_arr, h_arr, size, cudaMemcpyHostToDevice);

    // -------------------------------
    // Запуск версии с shared memory
    // -------------------------------
    auto startShared = high_resolution_clock::now();

    multiplyShared<<<blocksPerGrid, threadsPerBlock>>>(d_arr, factor, N);

    cudaDeviceSynchronize();

    auto endShared = high_resolution_clock::now();

    double timeShared =
        duration<double, milli>(endShared - startShared).count();

    // -------------------------------
    // Вывод результатов
    // -------------------------------
    cout << "Размер массива: " << N << endl;
    cout << "Глобальная память: " << timeGlobal << " мс" << endl;
    cout << "Разделяемая память: " << timeShared << " мс" << endl;

    // Освобождаем память
    cudaFree(d_arr);
    delete[] h_arr;

    return 0;
}

Overwriting task1.cu


In [12]:
!nvcc task1.cu -o task1
!./task1

Размер массива: 1000000
Глобальная память: 54.0846 мс
Разделяемая память: 0.072172 мс


# Задание 2

    Реализуйте CUDA-программу для поэлементного сложения двух массивов. Исследуйте
    влияние размера блока потоков на производительность программы. Проведите замеры
    времени для как минимум трёх различных размеров блока.

In [13]:
%%writefile task2.cu

#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

// ------------------------------------
// CUDA kernel для поэлементного сложения
// ------------------------------------
__global__ void vectorAdd(const int* A, const int* B, int* C, int n) {

    // Вычисляем глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверяем выход за границы массива
    if (idx < n) {
        // Каждый поток складывает один элемент
        C[idx] = A[idx] + B[idx];
    }
}

int main() {

    // Размер массива
    const int N = 1'000'000;
    size_t size = N * sizeof(int);

    // Выделяем память на CPU
    int* h_A = new int[N];
    int* h_B = new int[N];
    int* h_C = new int[N];

    // Инициализация входных массивов
    for (int i = 0; i < N; i++) {
        h_A[i] = i;
        h_B[i] = 2 * i;
    }

    // Выделяем память на GPU
    int *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    // Копируем данные с хоста на устройство
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // Массив размеров блоков для эксперимента
    int blockSizes[] = {128, 256, 512};

    cout << "Размер массива: " << N << " элементов\n";

    // Перебираем разные размеры блока
    for (int i = 0; i < 3; i++) {

        int threadsPerBlock = blockSizes[i];
        int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

        // Замер времени выполнения kernel
        auto start = high_resolution_clock::now();

        vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);

        // Ждём завершения выполнения kernel
        cudaDeviceSynchronize();

        auto end = high_resolution_clock::now();

        double timeMs =
            duration<double, milli>(end - start).count();

        // Вывод результатов
        cout << "Размер блока: " << threadsPerBlock
             << " | Время выполнения: "
             << timeMs << " мс" << endl;
    }

    // Копируем результат обратно на CPU (необязательно, но корректно)
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // Освобождение памяти
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    delete[] h_A;
    delete[] h_B;
    delete[] h_C;

    return 0;
}

Writing task2.cu


In [14]:
!nvcc task2.cu -o task2
!./task2

Размер массива: 1000000 элементов
Размер блока: 128 | Время выполнения: 7.72536 мс
Размер блока: 256 | Время выполнения: 0.004878 мс
Размер блока: 512 | Время выполнения: 0.001602 мс


# Задание 3

    Реализуйте CUDA-программу для обработки массива, демонстрирующую
    коалесцированный и некоалесцированный доступ к глобальной памяти. Сравните время
    выполнения обеих реализаций для массива размером 1 000 000 элементов.

In [15]:
%%writefile task3.cu

#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

// ------------------------------------
// Kernel с коалесцированным доступом
// ------------------------------------
__global__ void coalescedAccess(int* d_arr, int n) {

    // Вычисляем глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверка выхода за границы массива
    if (idx < n) {
        // Прямой поэлементный доступ (коалесцированный)
        d_arr[idx] = d_arr[idx] * 2 + 1;
    }
}

// ------------------------------------
// Kernel с некоалесцированным доступом
// ------------------------------------
__global__ void nonCoalescedAccess(int* d_arr, int n) {

    // Глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверяем, чтобы не выйти за границы
    if (idx < n) {
        // Некоалесцированный доступ:
        // каждый поток читает элементы с "скачком" по 32 элемента
        int offset = (idx * 32) % n;
        d_arr[offset] = d_arr[offset] * 2 + 1;
    }
}

int main() {

    // Размер массива
    const int N = 1'000'000;
    size_t size = N * sizeof(int);

    // Выделяем память на CPU
    int* h_arr = new int[N];

    // Инициализация массива
    for (int i = 0; i < N; i++) {
        h_arr[i] = i;
    }

    // Выделяем память на GPU
    int* d_arr;
    cudaMalloc(&d_arr, size);

    // Копируем данные с CPU на GPU
    cudaMemcpy(d_arr, h_arr, size, cudaMemcpyHostToDevice);

    // Параметры запуска
    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    // ------------------------------------
    // Коалесцированный доступ
    // ------------------------------------
    cudaMemcpy(d_arr, h_arr, size, cudaMemcpyHostToDevice); // восстанавливаем исходные данные
    auto startCoalesced = high_resolution_clock::now();

    coalescedAccess<<<blocksPerGrid, threadsPerBlock>>>(d_arr, N);
    cudaDeviceSynchronize();

    auto endCoalesced = high_resolution_clock::now();
    double timeCoalesced = duration<double, milli>(endCoalesced - startCoalesced).count();

    // ------------------------------------
    // Некоалесцированный доступ
    // ------------------------------------
    cudaMemcpy(d_arr, h_arr, size, cudaMemcpyHostToDevice); // восстанавливаем исходные данные
    auto startNonCoalesced = high_resolution_clock::now();

    nonCoalescedAccess<<<blocksPerGrid, threadsPerBlock>>>(d_arr, N);
    cudaDeviceSynchronize();

    auto endNonCoalesced = high_resolution_clock::now();
    double timeNonCoalesced = duration<double, milli>(endNonCoalesced - startNonCoalesced).count();

    // ------------------------------------
    // Вывод результатов
    // ------------------------------------
    cout << "Размер массива: " << N << " элементов" << endl;
    cout << "Коалесцированный доступ: " << timeCoalesced << " мс" << endl;
    cout << "Некоалесцированный доступ: " << timeNonCoalesced << " мс" << endl;

    // Освобождаем память
    cudaFree(d_arr);
    delete[] h_arr;

    return 0;
}

Writing task3.cu


In [16]:
!nvcc task3.cu -o task3
!./task3

Размер массива: 1000000 элементов
Коалесцированный доступ: 7.63436 мс
Некоалесцированный доступ: 0.073365 мс


  ## Вывод:
  
Для массива размером 1 000 000 элементов было проведено сравнение двух вариантов доступа к глобальной памяти GPU: коалесцированного и некоалесцированного.

При коалесцированном доступе время выполнения составило 7.63 мс. В этом случае потоки одного варпа обращаются к соседним ячейкам памяти, что позволяет GPU объединять обращения в меньшее количество транзакций памяти. Такой подход считается правильным и рекомендуется при работе с глобальной памятью.

При некоалесцированном доступе время выполнения оказалось 0.07 мс. Несмотря на то, что данный способ обращения к памяти считается менее эффективным с точки зрения архитектуры GPU, в рамках данной реализации он показал меньшее время выполнения. Это может быть связано с особенностями тестового ядра, простотой выполняемой операции и тем, что измеряемое время оказалось сильно зависимым от накладных расходов запуска ядра и оптимизаций компилятора.

В целом эксперимент наглядно демонстрирует, что способ доступа к глобальной памяти напрямую влияет на производительность CUDA-программ, однако для корректных выводов необходимо учитывать характер вычислений, объем реальной работы в ядре и влияние накладных расходов.



# Задание 4

    Для одной из реализованных в предыдущих заданиях CUDA-программ подберите
    оптимальные параметры конфигурации сетки и блоков потоков. Сравните
    производительность неоптимальной и оптимизированной конфигураций.

In [17]:
%%writefile task4.cu

#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

// -------------------------------
// CUDA kernel для поэлементного сложения массивов
// -------------------------------
__global__ void vectorAdd(int* a, int* b, int* c, int n) {
    // Глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверяем, чтобы индекс не вышел за границы массива
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

int main() {
    // Размер массива
    const int N = 1'000'000;

    // Выделяем память на хосте для массивов A, B и C
    int* h_a = new int[N];
    int* h_b = new int[N];
    int* h_c = new int[N];

    // Инициализация массивов случайными числами
    for (int i = 0; i < N; i++) {
        h_a[i] = rand() % 100;
        h_b[i] = rand() % 100;
    }

    // -------------------------------
    // Выделяем память на GPU
    // -------------------------------
    int *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, N * sizeof(int));
    cudaMalloc(&d_b, N * sizeof(int));
    cudaMalloc(&d_c, N * sizeof(int));

    // Копируем данные с CPU на GPU
    cudaMemcpy(d_a, h_a, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, N * sizeof(int), cudaMemcpyHostToDevice);

    // -------------------------------
    // Неоптимальная конфигурация
    // -------------------------------
    int threadsPerBlock1 = 64;                 // мало потоков в блоке
    int blocksPerGrid1 = (N + threadsPerBlock1 - 1) / threadsPerBlock1;

    auto start1 = high_resolution_clock::now();

    vectorAdd<<<blocksPerGrid1, threadsPerBlock1>>>(d_a, d_b, d_c, N);

    cudaDeviceSynchronize();
    auto end1 = high_resolution_clock::now();
    double timeNonOpt = duration<double, milli>(end1 - start1).count();

    // -------------------------------
    // Оптимальная конфигурация
    // -------------------------------
    int threadsPerBlock2 = 256;                // больше потоков на блок (GPU friendly)
    int blocksPerGrid2 = (N + threadsPerBlock2 - 1) / threadsPerBlock2;

    auto start2 = high_resolution_clock::now();

    vectorAdd<<<blocksPerGrid2, threadsPerBlock2>>>(d_a, d_b, d_c, N);

    cudaDeviceSynchronize();
    auto end2 = high_resolution_clock::now();
    double timeOpt = duration<double, milli>(end2 - start2).count();

    // -------------------------------
    // Копируем результат обратно на CPU (для проверки)
    // -------------------------------
    cudaMemcpy(h_c, d_c, N * sizeof(int), cudaMemcpyDeviceToHost);

    // -------------------------------
    // Выводим результаты
    // -------------------------------
    cout << "Размер массива: " << N << " элементов" << endl;
    cout << "Время выполнения (неоптимальная конфигурация): "
         << timeNonOpt << " мс" << endl;
    cout << "Время выполнения (оптимальная конфигурация): "
         << timeOpt << " мс" << endl;

    // Освобождаем память
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    delete[] h_a;
    delete[] h_b;
    delete[] h_c;

    return 0;
}


Writing task4.cu


In [18]:
!nvcc task4.cu -o task4
!./task4

Размер массива: 1000000 элементов
Время выполнения (неоптимальная конфигурация): 11.1259 мс
Время выполнения (оптимальная конфигурация): 0.002884 мс


# Вывод

  Для массива размером 1 000 000 элементов выбор конфигурации сетки и блоков потоков существенно влияет на производительность CUDA-программы. При неоптимальной конфигурации (64 потока на блок) время выполнения составило 11.13 мс, что связано с низкой загрузкой вычислительных ресурсов GPU и менее эффективным использованием аппаратного параллелизма.

  При оптимальной конфигурации (256 потоков на блок) время выполнения снизилось до 0.0029 мс. Это объясняется более полным использованием потоковых мультипроцессоров, лучшей скрываемостью задержек доступа к памяти и снижением накладных расходов на запуск блоков.

  Таким образом, корректный подбор параметров сетки и блоков является критически важным этапом оптимизации CUDA-программ и может давать выигрыш в производительности на порядки.